# ** Ok let's generate human mutability dictionaries** 

In [1]:
import pandas as pd
from Bio import AlignIO
import numpy

**generating the fastas dictionary ** 

In [2]:
list_chroms = ['chr' + str(i) for i in range(1, 23)]
fastas_dict = {}   # creating dictionary with fasta alignment, length of seq, 
for chrom in list_chroms:
    filename_tmp = "/scratch/research/projects/hsapiens/mutability/data/MichealsonData/Ready_data/Sequence/{c}.fa".format(c=chrom)
    fastas_dict[chrom] = []
    fastas_dict[chrom].append(AlignIO.read(open(filename_tmp),"fasta"))
    alignment_tmp = fastas_dict[chrom][0]
    fastas_dict[chrom].append(len(str(alignment_tmp[0].seq)))

** reading in the 100k mutations file ** 

In [144]:
mutations_lines = open('/scratch/research/projects/hsapiens/mutability/analysis/hg18_hg38_merged.txt').readlines()

**making and filling the mutation dictionary **

In [145]:
#making the blank dictionary --> rememebr to always reblank if the loop ran halfway then quit, as we a rappending in the for loop below below, not resetting. 
mutations_dictionary_ref_alt = {}
nucleotides = ["A", "T", "C", "G"]

list_fuck_ups = []

for base1 in nucleotides:
    for base2 in nucleotides:
        for base3 in nucleotides:
            triplet = base1+base2+base3
            mutations_dictionary_ref_alt[triplet] = [0,{}]
            for base_changing in nucleotides: 
                if base_changing != base2: 
                    mutations_dictionary_ref_alt[triplet][1][base1+base_changing+base3] = 0


                    
#filling the dictionary 
iterator = 0 

for line in mutations_lines: 
    row_split_tmp = line.split()
    if iterator >= 1: # skip the first lines 
        chrom_tmp = row_split_tmp[1]
        if chrom_tmp in list_chroms:
            site_number_tmp = row_split_tmp[2]
            site = [chrom_tmp, int(site_number_tmp)]
            #print(site)

            alignment = fastas_dict[site[0]][0]

            Ref_bp = row_split_tmp[8]
            New_bp = row_split_tmp[9]
            Ref_triplet= (str(alignment[0,site[1]-2:site[1]+1].seq)).upper()
            New_triplet = (str(alignment[0,site[1]-2])+str(New_bp)+str(alignment[0,site[1]])).upper()
            #print(Ref_bp); print(Ref_triplet); print(New_bp); print(New_triplet)

            if Ref_triplet == New_triplet: 
                #print(Ref_triplet +" "+New_triplet+" "+str(row_split_tmp[0])+"    "++"   "+(str(alignment[0,site[1]-2])+" "+Ref_bp+"   "+str(alignment[0,site[1]])+" "+New_bp)
                list_fuck_ups.append(str(row_split_tmp[0]))
            elif  len(Ref_triplet) == 3 and  len(New_triplet)==3 : 
                #ok now time to add to dictionary 
                mutations_dictionary_ref_alt[Ref_triplet][0]+=1
                mutations_dictionary_ref_alt[Ref_triplet][1][New_triplet] += 1 

        
        #break
    else: 
        print("skipped")
        #print("skipped"+line)
    iterator += 1
    
    
 

skipped


In [146]:
print(len(list_fuck_ups))

5215


In [147]:
print(mutations_dictionary_ref_alt)

{'AAT': [2304, {'ATT': 261, 'AGT': 1782, 'ACT': 261}], 'CTT': [1070, {'CAT': 148, 'CCT': 661, 'CGT': 261}], 'GAG': [759, {'GCG': 162, 'GTG': 186, 'GGG': 411}], 'TAC': [797, {'TGC': 571, 'TTC': 118, 'TCC': 108}], 'GCT': [1356, {'GGT': 256, 'GAT': 235, 'GTT': 865}], 'TCA': [1500, {'TGA': 363, 'TTA': 943, 'TAA': 194}], 'GGC': [1403, {'GTC': 333, 'GCC': 247, 'GAC': 823}], 'CTG': [1344, {'CGG': 277, 'CAG': 208, 'CCG': 859}], 'CAA': [1023, {'CGA': 602, 'CTA': 120, 'CCA': 301}], 'GTC': [584, {'GCC': 342, 'GAC': 137, 'GGC': 105}], 'TGC': [1314, {'TAC': 812, 'TTC': 321, 'TCC': 181}], 'TAA': [1240, {'TGA': 703, 'TTA': 270, 'TCA': 267}], 'TAG': [716, {'TTG': 102, 'TCG': 114, 'TGG': 500}], 'ACG': [2881, {'AAG': 131, 'AGG': 89, 'ATG': 2661}], 'GAA': [807, {'GTA': 160, 'GCA': 198, 'GGA': 449}], 'AAG': [1027, {'ACG': 230, 'AGG': 625, 'ATG': 172}], 'CAT': [2373, {'CCT': 329, 'CTT': 352, 'CGT': 1692}], 'CAC': [928, {'CCC': 194, 'CGC': 558, 'CTC': 176}], 'GAC': [505, {'GTC': 129, 'GCC': 85, 'GGC': 291}]

** ok now so the same for the non- mutant sites** 
- dont need too many, ~200k

In [148]:
non_mut_lines = open("/scratch/research/projects/hsapiens/mutability/data/MichealsonData/Ready_data/out4_testing_mutation_conditional.txt").readlines()

In [149]:
non_mut_dict = {}
for base1 in nucleotides:
    for base2 in nucleotides:
        for base3 in nucleotides:
            triplet = base1+base2+base3
            non_mut_dict[triplet] = 0

list_nonMut_fuckups = []

iterator = 0
for line in non_mut_lines: 
    if iterator >= 1:                        #skip the first line 
        line_split = line.split()
        triplet = line_split[2].upper()
        if triplet in non_mut_dict.keys(): 
            non_mut_dict[triplet] += 1
        else: 
            list_nonMut_fuckups.append(triplet)
    else: 
        print("skip")
    iterator += 1
    
    

skip


In [150]:
len(list_nonMut_fuckups)

6458

In [151]:
print(non_mut_dict)

{'AAT': 4769, 'CTT': 2906, 'GAG': 2408, 'TAC': 1905, 'GCT': 2675, 'TCA': 3457, 'GGC': 2586, 'CTG': 3317, 'CAA': 2792, 'GTC': 1420, 'TGC': 2706, 'TAA': 3105, 'TAG': 2003, 'ACG': 2966, 'GAA': 2657, 'AAG': 2977, 'CAT': 4074, 'CAC': 2363, 'GAC': 1433, 'TCC': 3021, 'TTG': 2748, 'GGT': 2778, 'AGC': 2758, 'TGA': 3370, 'ACA': 4616, 'AAC': 2327, 'AGT': 3370, 'ACT': 3370, 'CCC': 2655, 'GGA': 2950, 'TCG': 1887, 'TGT': 4426, 'GTG': 2370, 'GTT': 2240, 'TTA': 3405, 'AGG': 3367, 'CGA': 1953, 'ATC': 2225, 'GGG': 2536, 'CTA': 1942, 'CGT': 3042, 'GAT': 2277, 'GTA': 1976, 'TTC': 2791, 'CTC': 2363, 'GCC': 2519, 'CCA': 3159, 'TCT': 4287, 'AAA': 5291, 'CGG': 2617, 'CAG': 3247, 'ATA': 4417, 'TTT': 5468, 'TGG': 3194, 'TAT': 4486, 'CGC': 2266, 'GCG': 2354, 'ATT': 4532, 'ACC': 2684, 'CCG': 2576, 'GCA': 2822, 'CCT': 3435, 'AGA': 4145, 'ATG': 3986}


In [152]:
number_sites = 0
for key in non_mut_dict.keys(): 
    number_sites += non_mut_dict[key]
    

In [153]:
print(number_sites)

192767
